In [1]:
import pandas as pd

In [2]:
import sys
from pathlib import Path
repo_root = Path.cwd().resolve().parents[3]
print(f"Adding {repo_root} to sys.path")
sys.path.append(str(repo_root))
import utils

Adding /home/mrmath/sports_betting_empire/sports_betting_empire to sys.path


In [3]:
point_diff_spread_df = utils.fetch_common_stats_from_s3('americanfootball_nfl', 'point_diff_spread', 2018, 2025)

In [7]:
point_diff_spread_df

,Rk,Team,Date,Spread,PtDif,Day,G#,Week,Unnamed: 8_level_1,Opp,Result,Pts,PtsO,PC,vs. Line,Over/Under,OU Result,season
0,15,CIN,2018-12-09,17.0,-5,Sun,13,14,@,LAC,"L, 21-26",21,26,47,covered,48.5,under,2018
1,24,ARI,2018-11-11,16.5,-12,Sun,9,10,@,KAN,"L, 14-26",14,26,40,covered,49.5,under,2018
2,25,BUF,2018-09-23,16.5,21,Sun,3,3,@,MIN,"W, 27-6",27,6,33,covered,41.0,under,2018
3,43,NYJ,2018-12-30,14.5,-35,Sun,16,17,@,NWE,"L, 3-38",3,38,41,did not cover,46.5,under,2018
4,44,OAK,2018-12-30,14.5,-32,Sun,16,17,@,KAN,"L, 3-35",3,35,38,did not cover,52.5,under,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,4189,LAR,2025-11-02,-14.0,24,Sun,8,9,None,NOR,"W, 34-10",34,10,44,covered,44.0,push,2025
4250,4190,GNB,2025-10-12,-14.0,9,Sun,5,6,None,CIN,"W, 27-18",27,18,45,did not cover,45.0,push,2025
4251,4210,DEN,2026-01-04,-14.5,16,Sun,17,18,None,LAC,"W, 19-3",19,3,22,covered,37.5,under,2025
4252,4211,IND,2025-10-26,-14.5,24,Sun,8,8,None,TEN,"W, 38-14",38,14,52,covered,48.5,over,2025


In [74]:
point_diff_map = {}
for k, v in point_diff_spread_df.groupby('Team'):
    v = v.sort_values('Date')
    
    point_diff_3_ma = v['PtDif'].rolling(3).mean().shift(1)
    point_diff_5_ma = v['PtDif'].rolling(5).mean().shift(1)
    point_diff_3_sum = v['PtDif'].rolling(3).sum().shift(1)
    point_diff_5_sum = v['PtDif'].rolling(5).sum().shift(1)

    point_scored_3_ma = v['Pts'].rolling(3).mean().shift(1)
    point_scored_5_ma = v['Pts'].rolling(5).mean().shift(1)

    points_allowed_3_ma = v['PtsO'].rolling(3).mean().shift(1)
    points_allowed_5_ma = v['PtsO'].rolling(5).mean().shift(1)

    point_diff_map[k] = pd.DataFrame({
        'date': v['Date'],
        'point_diff_3_ma': point_diff_3_ma,
        'point_diff_5_ma': point_diff_5_ma,
        'point_diff_3_sum': point_diff_3_sum,
        'point_diff_5_sum': point_diff_5_sum,
        'point_scored_3_ma': point_scored_3_ma,
        'point_scored_5_ma': point_scored_5_ma,
        'points_allowed_3_ma': points_allowed_3_ma,
        'points_allowed_5_ma': points_allowed_5_ma,
    })


In [75]:
point_diff_spread_df[point_diff_spread_df['Over/Under'] == 48.5].sort_values('Date')    

,Rk,Team,Date,Spread,PtDif,Day,G#,Week,Unnamed: 8_level_1,Opp,Result,Pts,PtsO,PC,vs. Line,Over/Under,OU Result,season
101,813,DET,2018-09-16,6.0,-3,Sun,2,2,@,SFO,"L, 27-30",27,30,57,covered,48.5,over,2018
415,3538,SFO,2018-09-16,-6.0,3,Sun,2,2,None,DET,"W, 30-27",30,27,57,did not cover,48.5,over,2018
64,486,MIN,2018-09-27,7.5,-7,Thu,4,4,@,LAR,"L, 31-38",31,38,69,covered,48.5,over,2018
459,3861,LAR,2018-09-27,-7.5,7,Thu,4,4,None,MIN,"W, 38-31",38,31,69,did not cover,48.5,over,2018
242,2103,CHI,2018-10-21,1.0,-7,Sun,6,7,None,NWE,"L, 31-38",31,38,69,did not cover,48.5,over,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840,1019,MIA,2025-12-21,4.0,-24,Sun,15,16,None,CIN,"L, 21-45",21,45,66,did not cover,48.5,over,2025
4027,2420,ATL,2025-12-21,-2.5,7,Sun,15,16,@,ARI,"W, 26-19",26,19,45,covered,48.5,under,2025
4113,3134,CIN,2025-12-21,-4.0,24,Sun,15,16,@,MIA,"W, 45-21",45,21,66,covered,48.5,over,2025
3818,816,IND,2025-12-28,5.5,-6,Sun,16,17,None,JAX,"L, 17-23",17,23,40,did not cover,48.5,under,2025


In [82]:
point_diff_spread_train = []
for i in range(len(point_diff_spread_df)):
    team = point_diff_spread_df.loc[i, 'Team']
    date = point_diff_spread_df.loc[i, 'Date']
    opp = point_diff_spread_df.loc[i, 'Opp']

    team_data = None
    opp_data = None
    if team in point_diff_map:
        team_data = point_diff_map[team]
        team_date_data = team_data[team_data['date'] == date]
        if not team_date_data.empty:
            team_data = {
                'spread': point_diff_spread_df.loc[i, 'Spread'],

                'team': team,
                'date': date,
                'point_diff_3_ma': team_date_data['point_diff_3_ma'].iloc[-1],
                'point_diff_5_ma': team_date_data['point_diff_5_ma'].iloc[-1],
                'point_diff_3_sum': team_date_data['point_diff_3_sum'].iloc[-1],
                'point_diff_5_sum': team_date_data['point_diff_5_sum'].iloc[-1],
                'point_scored_3_ma': team_date_data['point_scored_3_ma'].iloc[-1],
                'point_scored_5_ma': team_date_data['point_scored_5_ma'].iloc[-1],
                'points_allowed_3_ma': team_date_data['points_allowed_3_ma'].iloc[-1],
                'points_allowed_5_ma': team_date_data['points_allowed_5_ma'].iloc[-1],
            }
    if opp in point_diff_map:
        opp_data = point_diff_map[opp]
        opp_date_data = opp_data[opp_data['date'] == date]
        if not opp_date_data.empty:

            opp_data = {
                'opp_point_diff_3_ma': opp_date_data['point_diff_3_ma'].iloc[-1],
                'opp_point_diff_5_ma': opp_date_data['point_diff_5_ma'].iloc[-1],
                'opp_point_diff_3_sum': opp_date_data['point_diff_3_sum'].iloc[-1],
                'opp_point_diff_5_sum': opp_date_data['point_diff_5_sum'].iloc[-1],
                'opp_point_scored_3_ma': opp_date_data['point_scored_3_ma'].iloc[-1],
                'opp_point_scored_5_ma': opp_date_data['point_scored_5_ma'].iloc[-1],
                'opp_points_allowed_3_ma': opp_date_data['points_allowed_3_ma'].iloc[-1],
                'opp_points_allowed_5_ma': opp_date_data['points_allowed_5_ma'].iloc[-1],
            }
    if team_data is not None and opp_data is not None:
        combined_data = {**team_data, **opp_data}
        point_diff_spread_train.append(combined_data)
    

In [83]:
point_diff_spread_train

[{'spread': np.float64(17.0),
  'team': 'CIN',
  'date': Timestamp('2018-12-09 00:00:00'),
  'point_diff_3_ma': np.float64(-10.666666666666666),
  'point_diff_5_ma': np.float64(-13.2),
  'point_diff_3_sum': np.float64(-32.0),
  'point_diff_5_sum': np.float64(-66.0),
  'point_scored_3_ma': np.float64(17.0),
  'point_scored_5_ma': np.float64(20.4),
  'points_allowed_3_ma': np.float64(27.666666666666668),
  'points_allowed_5_ma': np.float64(33.6),
  'opp_point_diff_3_ma': np.float64(12.333333333333334),
  'opp_point_diff_5_ma': np.float64(11.8),
  'opp_point_diff_3_sum': np.float64(37.0),
  'opp_point_diff_5_sum': np.float64(59.0),
  'opp_point_scored_3_ma': np.float64(33.333333333333336),
  'opp_point_scored_5_ma': np.float64(29.0),
  'opp_points_allowed_3_ma': np.float64(21.0),
  'opp_points_allowed_5_ma': np.float64(17.2)},
 {'spread': np.float64(16.5),
  'team': 'ARI',
  'date': Timestamp('2018-11-11 00:00:00'),
  'point_diff_3_ma': np.float64(-14.0),
  'point_diff_5_ma': np.float64(-

In [84]:
point_diff_spread_train_df = pd.DataFrame(point_diff_spread_train)

In [85]:
point_diff_spread_train_df=point_diff_spread_train_df.sort_values('date').dropna(subset=['date'])

In [86]:
point_diff_map['PHI']

,date,point_diff_3_ma,point_diff_5_ma,point_diff_3_sum,point_diff_5_sum,point_scored_3_ma,point_scored_5_ma,points_allowed_3_ma,points_allowed_5_ma
278,2018-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,2018-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,2018-09-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,2018-09-30,1.333333,NaN,4.0,NaN,19.666667,NaN,18.333333,NaN
366,2018-10-07,-1.666667,NaN,-5.0,NaN,21.333333,NaN,23.000000,NaN
...,...,...,...,...,...,...,...,...,...
4016,2025-12-08,-1.666667,3.2,-5.0,16.0,17.333333,20.0,19.000000,16.8
4236,2025-12-14,-5.000000,-1.0,-15.0,-5.0,18.333333,16.2,23.333333,17.2
4173,2025-12-20,6.333333,4.6,19.0,23.0,21.666667,20.4,15.333333,15.8
3950,2025-12-28,13.000000,5.4,39.0,27.0,26.333333,23.0,13.333333,17.6


In [87]:
point_diff_spread_train_df

,spread,team,date,point_diff_3_ma,point_diff_5_ma,point_diff_3_sum,point_diff_5_sum,point_scored_3_ma,point_scored_5_ma,points_allowed_3_ma,points_allowed_5_ma,opp_point_diff_3_ma,opp_point_diff_5_ma,opp_point_diff_3_sum,opp_point_diff_5_sum,opp_point_scored_3_ma,opp_point_scored_5_ma,opp_points_allowed_3_ma,opp_points_allowed_5_ma
278,-1.0,PHI,2018-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,1.0,ATL,2018-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,-1.0,CIN,2018-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,-1.0,TEN,2018-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,-3.5,LAC,2018-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,8.0,NYJ,2026-01-04,-27.666667,-20.8,-83.0,-104.0,12.000000,14.6,39.666667,35.4,2.000000,6.0,6.0,30.0,23.333333,27.0,21.333333,21.0
3736,10.5,GNB,2026-01-04,-10.333333,-3.4,-31.0,-17.0,22.000000,25.0,32.333333,28.4,8.000000,5.8,24.0,29.0,24.333333,20.8,16.333333,15.0
3756,8.0,CLE,2026-01-04,-8.000000,-8.8,-24.0,-44.0,12.000000,14.6,20.000000,23.4,7.666667,7.2,23.0,36.0,27.333333,29.6,19.666667,22.4
3711,14.5,LAC,2026-01-04,5.333333,7.2,16.0,36.0,22.000000,23.8,16.666667,16.6,0.333333,1.8,1.0,9.0,24.666667,25.0,24.333333,23.2


In [88]:
point_diff_spread_train_df.to_csv('point_diff_spread_train.csv', index=False)